In [ ]:
%%bash
nrnivmodl

In [ ]:
# Download NEURON: http://www.neuron.yale.edu/neuron/download
# Download PyNeuronToolbox: https://github.com/ahwillia/PyNeuron-Toolbox

from neuron import h
import numpy as np
import math
import pylab as plt
import scipy.linalg
from PyNeuronToolbox.record import ez_record,ez_convert
from PyNeuronToolbox.morphology import shapeplot,shapeplot_animate
from mpl_toolkits.mplot3d import Axes3D
from JSAnimation import IPython_display
from matplotlib import animation
from matplotlib.pyplot import cm
import warnings
warnings.filterwarnings('ignore', category=SyntaxWarning)
%matplotlib inline
np.random.seed(123456789)

DC = 10 # diffusion coeff [microns^2 s^-1]

# Load morphology and other stuff
# --> SegLists: soma[2], dend[74], dend_5[37], apic[42], axon[1]
# --> Files from Migliore & Migliore (2012)
# --> CA1 pyramidal neuron
h.load_file('stdrun.hoc')
h.xopen('ri06.hoc')
h.xopen('fixnseg.hoc')
h.xopen('5a_nogui.hoc')
h.tstop = 700.0

In [ ]:
from PyNeuronToolbox.morphology import dist_between,allsec_preorder

def dist_to_soma(segment):
    return dist_between(h,h.soma[0](0.5),segment)

#seglist in pre-order
sec_list = allsec_preorder(h)
seg_list = []
for sec in sec_list:
    locs = np.linspace(0,1,sec.nseg+2)[1:-1]
    for loc in locs:
        seg_list.append(sec(loc))
N = len(seg_list)
n = N

In [ ]:
for i in range(len(sec_list)):
    sec = sec_list[i]
    print(f"{i}: {sec.name()}, nseg={sec.nseg}, L={sec.L}, or={sec.orientation()}, hname={sec.hname()}")

In [ ]:
import re
import pandas as pd
p = re.compile("^([^\\[]+)\\[.+")
tps = [p.sub("\\1",seg.sec.name()) for seg in seg_list]
pd.Series(tps).drop_duplicates().tolist()

In [ ]:
def sushi_system(a,b,c,d,l):
    """
    Returns a matrix A, such that dx/dt = A*x
    
    N = # of compartments
    A is (2N x 2N) matrix
    x is (2N x 1) vector.
      The first N elements correspond to concentrations of u (molecules in transit)
      The second half correspond to concentrations of u-star (active molecules)
    The trafficking rate constants along the microtubules are given by the vectors "a" and "b"
    The rate constants for u turning into u* is given by the vector "c"
    The rate constants for u* turning into u is given by the vector "d"
    The rate constants for the degradation of u* is given by the vector "l"
    """
    # number of compartments
    N = len(l)
    
    ## State-space equations
    #  dx/dt = Ax + Bu
    A = np.zeros((2*N,2*N))

    # Trafficking along belt
    # Iterative traversal of dendritic tree in pre-order
    i = 0
    section = None
    parentStack = [(None,h.soma[0])]
    while len(parentStack)>0:
        # Get next section to traverse
        #  --> p is parent index, section is h.Section object
        (p,section) = parentStack.pop()
        
        # Trafficking to/from parent
        if p is not None:
            # Out of parent, into child
            ai = a.pop()
            A[p,p] += -ai
            A[i,p] += ai
            # Into parent, out of child
            bi = b.pop()
            A[p,i] += bi
            A[i,i] += -bi
        
        # visit all segments in compartment
        for (j,seg) in enumerate(section):
            # Deal with out/into rates within compartment, just tridiag matrix
            if j>0:
                # Out of parent, into child
                ai = a.pop()
                A[i-1,i-1] += -ai
                A[i,i-1] += ai
                # Into parent, out of child
                bi = b.pop()
                A[i-1,i] += bi
                A[i,i] += -bi
            # move onto next compartment
            i += 1
        
        # now visit children in pre-order
        child_list = list(h.SectionRef(sec=section).child)
        if child_list is not None:
            child_list.reverse()
        for c_sec in child_list:
            parentStack.append([i-1,c_sec]) # append parent index and child
    
    # Trafficking off the belt
    for i in range(N):
        A[i,i] += -c[i]
        A[i+N,i] += c[i]
    
    # Reattachment to belt
    #for i in range(N):
    #    # reattachment
    #    A[i, i + N] += d[i]
    #    A[i + N, i + N] += -d[i]

    # Degradation after being taken off the belt
    for i in range(N):
        A[i+N,i+N] = -l[i]
    
    return A

In [ ]:
def trafficking_solution(utarg):
    """ Solve the problem by tuning trafficking rates, like Figs 1 and 2. """
    x = []
    
    # Iterative traversal of dendritic tree in pre-order
    i = 0
    section = None
    parentStack = [(None,h.soma[0])]
    while len(parentStack)>0:
        # Get next section to traverse
        #  --> p is parent index, section is h.Section object
        (p,section) = parentStack.pop()
        
        # Trafficking to/from parent
        if p is not None:
            mp = utarg[p] # concentration in parent
            mc = utarg[i] # concentration in child
            x.insert(0,mp/mc)
        
        # visit all segments in compartment
        for (j,seg) in enumerate(section):
            # Deal with out/into rates within compartment, just tridiag matrix
            if j>0:
                mp = utarg[i-1]
                mc = utarg[i]
                x.insert(0,mp/mc)
                
            # move onto next compartment
            i += 1
        
        # now visit children in pre-order
        child_list = list(h.SectionRef(sec=section).child)
        if child_list is not None:
            child_list.reverse()
        for c_sec in child_list:
            parentStack.append([i-1,c_sec]) # append parent index and child
    
    # return calculated guesses (flip, up/down since get_deriv pops from start)
    return np.array(x)

def get_sys_matrix(utarg,F=0.5,Ctau=1e-3,dscale=0.1,dv=1e-7):
    # F is a mixing factor between 0 and 1
    K = np.sum(utarg)/n
    x = trafficking_solution(F*utarg + (1-F)*K)
    a = (1/(1+x))
    a = list(a)
    b = list((1/(1+x**-1)))
    l = list(np.ones(n)*dv)
    c = list(Ctau*utarg/(F*utarg + (1-F)*K))
    d = list([ci * dscale for ci in c])
    A = sushi_system(a,b,c,d,l)
    return A

In [ ]:
# matrix exponential used to solve linear system
from scipy.linalg import expm

# initial condition, u starts only in soma and is trafficked out to dendrites
u0 = np.zeros(2*N)
u0[0] = 1.0  # compartment 0 is the soma

In [ ]:
def solve_u(u0,w,V,Vinv,t):
    D = np.diag(np.exp(w*t))          # diagonal matrix exponential
    PHI = np.real(V.dot(D.dot(Vinv))) # state transition matrix
    return PHI.dot(u0)                # calculate u(t)

def sim_time(A,u0,time,nframes=10):
    # Run a simulation (log time)
    # --> this is a linear system; thus, matrix exponential provides exact solution
    utrace = [u0]
    w,V = scipy.linalg.eig(A)
    Vinv = np.linalg.inv(V)
    t = np.logspace(-0.5,math.log10(time),nframes)
    for t_ in t: utrace.append(solve_u(u0,w,V,Vinv,t_))
    return np.array(utrace).T


def run_sim(A,nframes=10):
    # Run a simulation (log time)
    # --> this is a linear system; thus, matrix exponential provides exact solution
    utrace = [u0]
    t = np.logspace(-0.5,8.5,nframes)
    w,V = scipy.linalg.eig(A)
    Vinv = np.linalg.inv(V)
    t = np.logspace(-0.5,8.5,nframes)
    for t_ in t: utrace.append(solve_u(w,V,Vinv,t_))
    return np.array(utrace).T

def animate_sim(climits=[0,0.0035]):
    u = utrace[n:,:]
    fig = plt.figure(figsize=(8,8))
    shapeax = plt.subplot(111, projection='3d')
    lines = shapeplot(h,shapeax,order='pre',lw=2)
    shapeax.view_init(elev=80,azim=-90)
    plt.title('distribution of active/detached cargo',fontweight='bold',fontsize=14)
    plt.axis('off')

    # Hack to get a colorbar
    # http://stackoverflow.com/questions/8342549/matplotlib-add-colorbar-to-a-sequence-of-line-plots
    sm = plt.cm.ScalarMappable(cmap=plt.cm.cool, norm=plt.Normalize(climits[0],climits[1])) 
    #sm._A = []
    #plt.colorbar(sm, shrink=0.5)
    #plt.colorbar(sm)
    plt.tight_layout()

    anim_func = shapeplot_animate(u.T,lines,u.shape[1],clim=climits,cmap=cm.cool)
    return animation.FuncAnimation(fig, anim_func, frames=u.shape[1], interval=5e2, blit=True)

def snap_ss(name,clim=[0,0.0035],cmap=plt.cm.cool):
    u = utrace[n:,:]
    fig = plt.figure(figsize=(8,8))
    shapeax = plt.subplot(111, projection='3d')
    lines = shapeplot(h,shapeax,cvals=u[:,-1],clim=clim,\
                      cmap=cmap,order='pre',lw=2)
    shapeax.view_init(elev=80,azim=-90)
    plt.title('steady-state',fontweight='bold',fontsize=14)

    # colorbar
    sm = cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(clim[0],clim[1])) 
    #sm = plt.cm.ScalarMappable(cmap=plt.cm.cool, norm=plt.Normalize(clim[0],clim[1])) 
    #sm._A = []
    #plt.colorbar(sm, shrink=0.5)
    #plt.colorbar(sm,cax=shapeax)
    #plt.colorbar()
    plt.tight_layout()
    plt.axis('off')
    #plt.savefig(name+'.svg')

Read Experimental Data
----

In [ ]:
tdf=pd.read_csv('../data/seg_mapping.csv')
abbCA1=tdf['abb']
abbT={}
segIdx={}
for i in range(N):
    abbT[abbCA1[i]] = 1+ abbT.get(abbCA1[i],0)
    ll=segIdx.get(abbCA1[i],[])
    ll.append(i)
    segIdx[abbCA1[i]] = ll

(abbCA1,abbT,segIdx.keys(),{k:len(segIdx.get(k)) for k in segIdx.keys()})

In [ ]:
ks=[i for i in segIdx.keys()]
(ks[0],segIdx[ks[0]])

In [ ]:
sec=seg_list[0]
sec.sec.name()

In [ ]:
sname=[]
hname=[]
for i in range(len(seg_list)):
    sec = seg_list[i].sec
    sname.append(sec.name())
    hname.append(sec.hname())
#(sname,hname)

In [ ]:
for i in segIdx[ks[0]]:
    sec = seg_list[i].sec
    print(f"{i}: {sec.name()}, nseg={sec.nseg}, L={sec.L}, or={sec.orientation()}, hname={sec.hname()}")

In [ ]:
expD=pd.read_csv('../data/CA1_gradient.csv')
subreg = ['CA1so', 'CA1sr', 'CA1slm']
expD

In [ ]:
import sys
sys.path.append('../')
import sushibelt
import time

In [ ]:
utarg = np.ones(len(seg_list))
for i in range(expD.shape[0]):
    abb = expD['Abbreviation'][i]
    sidx= segIdx[abb]
    utarg[sidx] *= i
u1 = np.concatenate((utarg,utarg))
utrace = [u1,u1]
utrace = np.array(utrace).T
snap_ss(f"segment_distribution",clim=[0,max(utarg)],cmap=plt.cm.YlOrBr)
max(utarg)

In [ ]:
utarg = np.ones(len(seg_list))
for i in range(expD.shape[0]):
    abb = expD['Abbreviation'][i]
    
    sidx= segIdx[abb]
    utarg[sidx] *= [j for j in range(len(subreg)) if subreg[j]== expD['Subregion'][i]][0]
u1 = np.concatenate((utarg,utarg))
utrace = [u1,u1]
utrace = np.array(utrace).T
snap_ss(f"subregion_distribution",clim=[0,max(utarg)],cmap=plt.cm.YlOrBr)
max(utarg)

Day 0 3 month
------

In [ ]:
cname='D0M3'
utarg = np.ones(len(seg_list))
for i in range(expD.shape[0]):
    abb = expD['Abbreviation'][i]
    sidx= segIdx[abb]
    utarg[sidx] *= expD[f"{cname}_MEAN"][i]/len(sidx)
u1 = np.concatenate((utarg,utarg))
utrace = [u1,u1]
utrace = np.array(utrace).T
snap_ss(f"reg20_{cname}_target",clim=[0,max(utarg)],cmap=plt.cm.YlOrBr)
max(utarg)

In [ ]:
x=np.array(range(expD.shape[0]),dtype=int)
nobsD0 = np.array(expD[f"{cname}_MEAN"])#/np.sum(expD[f"{cname}_MEAN"])
plt.plot(x,nobsD0,label=cname)
plt.show()

In [ ]:
cname7='D7M3'
obsD = np.ones(len(seg_list))
for i in range(expD.shape[0]):
    abb = expD['Abbreviation'][i]
    sidx= segIdx[abb]
    obsD[sidx] *= expD[f"{cname7}_MEAN"][i]/len(sidx)
nobsD = np.array(expD[f"{cname7}_MEAN"])#/np.sum(expD[f"{cname}_MEAN"])
uD = np.concatenate((obsD,obsD))
utrace = [uD,uD]
utrace = np.array(utrace).T
max(obsD)

In [ ]:
x=np.array(range(expD.shape[0]),dtype=int)
cname0='D0M3'
cname7='D7M3'

plt.plot(x,np.array(expD[f"{cname7}_MEAN"])/np.sum(expD[f"{cname0}_MEAN"]),label=cname7)
plt.plot(x,np.array(expD[f"{cname0}_MEAN"])/np.sum(expD[f"{cname0}_MEAN"]),label=cname0)
plt.ylabel('Value')
plt.xlabel('Region number')
# show a legend on the plot
plt.legend()
plt.show()
#plt.savefig('bestFitCA1_20reg_1dv.png')
#plt.savefig('bestFitCA1_20reg_1dv.pdf')
#plt.savefig('bestFitCA1_20reg_1dv.svg')


In [ ]:
x=np.array(range(expD.shape[0]),dtype=int)
cname0='D0M3'
cname7='D7M3'

plt.plot(x,np.array(expD[f"{cname7}_MEAN"])/np.sum(expD[f"{cname7}_MEAN"]),label=cname7)
plt.plot(x,np.array(expD[f"{cname0}_MEAN"])/np.sum(expD[f"{cname0}_MEAN"]),label=cname0)
plt.ylabel('Value')
plt.xlabel('Region number')
# show a legend on the plot
plt.legend()
plt.show()
#plt.savefig('bestFitCA1_20reg_1dv.png')
#plt.savefig('bestFitCA1_20reg_1dv.pdf')
#plt.savefig('bestFitCA1_20reg_1dv.svg')


In [ ]:
diF=abs(uD-u1)
utrace = [diF,diF]
utrace = np.array(utrace).T
(min(diF),max(diF))

In [ ]:
cname7='D7M3'
obsD = np.ones(len(seg_list))
resMo = np.zeros(expD.shape[0])
resFo = np.zeros(expD.shape[0])
for i in range(expD.shape[0]):
    abb = expD['Abbreviation'][i]
    sidx= segIdx[abb]
    obsD[sidx] *= expD[f"{cname7}_MEAN"][i]
    resMo[i] = np.sum(utrace[sidx,-1])
    resFo[i] = np.sum(utrace[[j+n for j in sidx],-1])
nobsD = np.array(expD[f"{cname7}_MEAN"])/np.sum(expD[f"{cname7}_MEAN"])
nobsDnn = np.array(expD[f"{cname7}_MEAN"])/np.sum(expD["D0M3_MEAN"])
nresM = resMo/np.sum(resMo)
nresF = resFo/np.sum(resFo)
(nobsD,nresM,nresF)

In [ ]:
i=0
abb = expD['Abbreviation'][i]
sidx = segIdx[abb]
(i,abb,sidx)

In [ ]:
itarg = np.ones(len(seg_list), dtype=int)
for i in range(expD.shape[0]):
    abb = expD['Abbreviation'][i]
    sidx = segIdx[abb]
    itarg[sidx] *= i+5
utarg = [par[j] for j in itarg]
tgAbb = []
tgFnd = []
for i in itarg:
    if i>4:
        tgFnd.append(f'found:{i-5}')#
        tgAbb.append(expD['Abbreviation'][i-5])
    else:
        tgAbb.append('none')
        tgFnd.append('none')
itargDF=pd.DataFrame({'itarg':itarg,'utarg': utarg,'name':sname,'hname':hname,'abbCA1':abbCA1,'tgFnd':tgFnd,'tgAbb':tgAbb})
itargDF

In [ ]:
lowb=np.array([0,-18,0,-18,-18,1.0e-07,1.0e-07,1.0e-07,1.0e-07,1.0e-07,1.0e-07,1.0e-07,1.0e-07,1.0e-07,1.0e-07,1.0e-07,1.0e-07,1.0e-07,1.0e-07,1.0e-07,1.0e-07,1.0e-07,1.0e-07,1.0e-07,1.0e-07])
upbga=np.array([1,-1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1])
Ndim=len(lowb)
(Ndim,len(lowb),len(upbga))

In [ ]:
bgSignal = 1e-5
cname0='D0M3'
d0w = -1 * np.ones(N)
for i in range(expD.shape[0]):
    abb = expD['Abbreviation'][i]
    sidx= segIdx[abb]
    d0w[sidx] *= -1*expD[f"{cname0}_MEAN"][i]/len(sidx)
for i in range(N):
    if d0w[i]<0:
        d0w[i] = bgSignal
dinit = d0w/np.sum(d0w)

#cname='D7W3'
cname7='D7M3'
d7w = -1*np.ones(N)
for i in range(expD.shape[0]):
    abb = expD['Abbreviation'][i]
    sidx= segIdx[abb]
    d7w[sidx] *= -1 * expD[f"{cname7}_MEAN"][i]/len(sidx)
for i in range(N):
    if d7w[i]<0:
        d7w[i] = bgSignal
target = np.array(expD[f"{cname7}_MEAN"])/np.sum(expD[f"{cname0}_MEAN"]) #norm target to Day0 sum to take into accound degradation
targSD = np.array(expD[f"{cname7}_SD"])/np.sum(expD[f"{cname0}_MEAN"]) #measurement errors
tnorm = np.sum(target ** 2)
day7 = 7 * 24 * 3600 # final time point

itarg = np.ones(N, dtype=int)
for i in range(expD.shape[0]):
    abb = expD['Abbreviation'][i]
    sidx = segIdx[abb]
    itarg[sidx] *= (i+5)

def calcUtrace(par,delta=bgSignal,nframes=10):
    F = par[0]
    Ctau = 10 ** par[1]
    mProp = par[2]
    dvA = par[3]
    dvB = par[4]
    dv = np.zeros(N)
    utarg = delta*np.ones(N)
    for k in range(N):
        if itarg[k] > 4:
            utarg[k] = par[itarg[k]]
            dv[k] = (10 ** dvA) + (10 ** dvB)*utarg[k]
    utarg /= np.sum(utarg)
    K = np.sum(utarg) / N
    x = trafficking_solution(F * utarg + (1 - F) * K)
    a = (1 / (1 + x))
    a = list(a)
    b = list((1 / (1 + x ** -1)))
    l = list(dv)
    c = list(Ctau * utarg / (F * utarg + (1 - F) * K))
    d = list(np.zeros(N))
    A = sushi_system(a, b, c, d, l)
    u0 = np.concatenate((mProp * dinit, (1 - mProp) * dinit))
    utrace = sim_time(A, u0, day7,nframes=nframes)
    return utrace

#par = [0.111784701917933,-1,0.584647952749801,-5.40926182202759,-7.58373647291078,0.93075347720533,0.150119155946245,0.250026616923097,0.247923892623914,1,0.829698078092076,0.799680386796054,0.435204401984173,0.776990953492751,0.658564397472255,0.669466722069782,0.13217494651507,0.452337127435389,0.818398235362628,0.287266237055001,0.693374804035749,0.387600533946115,0.860396244028168,0.793857689742176,0.00234423876609767]
#cost=0.2289827
#par = [0.141527769926587,-1.04440055844237,0.381706990604462,-5.43766625150405,-12.234744035647,1,0.164098437336494,0.272405308568442,0.427804635567134,0.812746641751752,0.919149761897559,0.672064846552212,0.149965791068772,0.539839422213906,0.812188937409397,0.806685239524521,0.165041822978698,0.529423937850048,0.828384035574742,0.221525313745652,0.676304529167494,0.634182580029409,0.814899037032692,0.790009814378913,0.0884319910464033]
#cost=0.1914477
#par = [1.0,-18.0,0.339665263904274,-18.0,-0.1056091212860899,1.0,0.06274608035463802,4.4574543322602384e-07,0.02193672327975435,0.0870196802604857,8.738149616920519e-07,1.0,0.23641992030781905,0.20499095398769174,3.3139951083305927e-07,0.0264354594647535,1.0601425384202188e-07,0.018642526550405554,1.3229426155908984e-07,0.321659768251798,1.256853193752937e-06,0.9999991575763989,0.9008918582308576,1.0002936595814933e-07,1.0]
#cost = 0.00972634
#par = [0.18124010722657255,-18.0,0.4770849437408547,-18.0,-7.528208406834074,0.6335729107600226,0.28276999275111847,0.558910113544302,1.0,0.3929830308454687,0.2992123958197709,0.753091076326252,1.0,0.17238067898531845,1.0,1e-07,1.0,0.04290036974800929,0.9692956552909402,1.0,0.5346212757781438,1.0,0.28625682786833306,1e-07,1e-07]
#cost = 0.02659
#par = [0.32023059450279123,-6.327308855004608,0.627385538130974,-18.0,-17.999431588729976,0.06854654347167044,0.060486839296889444,0.04484900714349323,0.03490854158366522,0.00024252729610862994,0.32943753444660906,0.4029203029951907,0.12801603593190886,0.14871930210853104,0.14171491277653797,0.18024101228415457,0.18628112282529372,0.23357327869041905,0.4302044118823936,0.5008667022509936,0.0038157043176590766,0.2007104314288199,0.22494314014617123,0.31189759389623783,0.9999999999990268]
#cost = 0.001142
#par = [0.01385481065982917,-6.226265838840085,0.6709771115511489,-17.995913305482574,-16.696101588995024,0.21868548101204954,0.06211405206769383,0.048882353452234434,0.046632773486508856,1.9401449712242143e-06,0.7864443640945032,0.5632548318074286,0.14915702983161863,0.1491589398028194,0.13322800132469398,0.15668388533325328,0.17435550169553826,0.20436446923690443,0.37541096611836217,0.5260391371129066,0.043995168941356404,0.1970021740399707,0.19622524557390722,0.2758337024352678,0.9999999999613025]
#cost = 1.51e-09
#par = [0.0,-2.847372550934951,0.20430623530833164,-5.754630055624563,-5.8321723784857635,-6.0649088262203215,0.9999999999999993,0.9550522528085269,0.001]
#cost = 0.00010140231658948862
par = [0.8861278052764772,-5.358123919558814,0.3127455791698551,-5.7356821635860165,-17.999032750284783,0.21382364663804718,0.06074280874523973,0.04781382417616123,0.04562340830020193,1.0000000000039887e-07,0.7733439929081787,0.5543045057407645,0.1468913931288651,0.14710118884157364,0.1316203146580693,0.15499357159791044,0.17296720163305693,0.20287137824434023,0.3735878952746845,0.5243019440900127,0.043864102155039865,0.1966480252602025,0.19591970216660096,0.2755074889018492,0.9999999999999992]
cost = 3.331839554992909e-11
mProp = par[2]
utrace = calcUtrace(par)
resM, resF = sushibelt.aggregate_segments(utrace[:, -1], segIdx, expD['Abbreviation'], fun=np.sum)
bpar = par

In [ ]:
np.log(2)/(10 ** par[3])/3600

In [ ]:
{'F' : par[0],
'Ctau' : 10 ** par[1],
'mProp' : par[2],
'dvA': 10 ** par[3],
'dvB': 10 ** par[4],
'demand_CA1so_1' : par[5],
'demand_CA1so_2' : par[6],
'demand_CA1so_3' : par[7],
'demand_CA1so_4' : par[8],
'demand_CA1so_5' : par[9],
'demand_CA1sr_1' : par[10],
'demand_CA1sr_2' : par[11],
'demand_CA1sr_3' : par[12],
'demand_CA1sr_4' : par[13],
'demand_CA1sr_5' : par[14],
'demand_CA1sr_6' : par[15],
'demand_CA1sr_7' : par[16],
'demand_CA1sr_8' : par[17],
'demand_CA1sr_9' : par[18],
'demand_CA1sr_10' : par[19],
'demand_CA1slm_1' : par[20],
'demand_CA1slm_2' : par[21],
'demand_CA1slm_3' : par[22],
'demand_CA1slm_4' : par[23],
'demand_CA1slm_5' : par[24],
'sum_nobsDnn' : np.sum(nobsDnn),
'sum_nobsDnn^2' : np.sum(nobsDnn ** 2),
'sum_resM' : np.sum(resM),
'sum_resF' : np.sum(resF),
'sum_tot' : np.sum(resF)+np.sum(resM),
'err' : np.sum((resF/(1-mProp) - target) ** 2),
'err_norm' : np.sum((resF/(1-mProp) - target) ** 2)/tnorm,
'chi2' : np.sum(((resF/(1-mProp) - target)/targSD) ** 2),
'cost' : cost
}

In [ ]:
delta=bgSignal
F = par[0]
Ctau = 10 ** par[1]
mProp = par[2]
dvA = par[3]
dvB = par[4]
dv = np.zeros(N)
utarg = delta*np.ones(N)
for k in range(N):
    if itarg[k] > 4:
        utarg[k] = par[itarg[k]]
        dv[k] = (10 ** dvA) + (10 ** dvB)*utarg[k]
utarg /= np.sum(utarg)
K = np.sum(utarg) / N
x = trafficking_solution(F * utarg + (1 - F) * K)
a = (1 / (1 + x))
aM, aF = sushibelt.aggregate_segments(np.concatenate((list(a),list(a))), segIdx, expD['Abbreviation'], fun=np.sum)
b = list((1 / (1 + x ** -1)))
bM, bF = sushibelt.aggregate_segments(np.concatenate((b,b)), segIdx, expD['Abbreviation'], fun=np.sum)
l = list(dv)
dM, dF = sushibelt.aggregate_segments(np.concatenate((l,l)), segIdx, expD['Abbreviation'], fun=np.sum)
c = list(Ctau * utarg / (F * utarg + (1 - F) * K))
cM, cF = sushibelt.aggregate_segments(np.concatenate((c,c)), segIdx, expD['Abbreviation'], fun=np.sum)
uM, uF = sushibelt.aggregate_segments(np.concatenate((utarg,utarg)), segIdx, expD['Abbreviation'], fun=np.sum)
df=pd.DataFrame([aM,bM,cM,dM,uM,par[5:]],columns=expD['Abbreviation'],index=['a','b','c','d','utarg','demand'])
#df.to_csv('CA1_20reg_1dv_experiment.sum.heatmap.csv')
df

In [ ]:
delta=bgSignal
F = par[0]
Ctau = 10 ** par[1]
mProp = par[2]
dvA = par[3]
dvB = par[4]
dv = np.zeros(N)
utarg = delta*np.ones(N)
for k in range(N):
    if itarg[k] > 4:
        utarg[k] = par[itarg[k]]
        dv[k] = (10 ** dvA) + (10 ** dvB)*utarg[k]
utarg /= np.sum(utarg)
K = np.sum(utarg) / N
x = trafficking_solution(F * utarg + (1 - F) * K)
a = (1 / (1 + x))
aM, aF = sushibelt.aggregate_segments(np.concatenate((list(a),list(a))), segIdx, expD['Abbreviation'], fun=np.mean)
b = list((1 / (1 + x ** -1)))
bM, bF = sushibelt.aggregate_segments(np.concatenate((b,b)), segIdx, expD['Abbreviation'], fun=np.mean)
l = list(dv)
dM, dF = sushibelt.aggregate_segments(np.concatenate((l,l)), segIdx, expD['Abbreviation'], fun=np.mean)
c = list(Ctau * utarg / (F * utarg + (1 - F) * K))
cM, cF = sushibelt.aggregate_segments(np.concatenate((c,c)), segIdx, expD['Abbreviation'], fun=np.mean)
uM, uF = sushibelt.aggregate_segments(np.concatenate((utarg,utarg)), segIdx, expD['Abbreviation'], fun=np.mean)
df=pd.DataFrame([aM,bM,cM,dM,uM,par[5:]],columns=expD['Abbreviation'],index=['a','b','c','d','utarg','demand'])
#df.to_csv('CA1_20reg_1dv_experiment.mean.heatmap.csv')
df

In [ ]:
plt.scatter(uM,par[5:])
plt.title("Utarg vs par") 
plt.ylabel('Utarg')
plt.xlabel('demand')
plt.show()


In [ ]:
x=np.array(range(expD.shape[0]),dtype=int)
plt.plot(x,uM,label='uM')
#plt.plot(x,nresM,label='mobile O')
#plt.plot(x,nresF,label='detached O')
plt.plot(x,par[5:]/np.sum(par[5:]),label='par')
plt.ylabel('Value')
plt.xlabel('Region number')
# show a legend on the plot
plt.legend()
plt.title("Utarg vs par") 
# function to show the plot
plt.show()


In [ ]:
x=np.array(range(expD.shape[0]),dtype=int)
plt.plot(x,target,'o-',label=cname7)
plt.plot(x,uM/np.sum(uM),label='uM')
plt.plot(x,np.array(expD[f"{cname0}_MEAN"])/np.sum(expD[f"{cname0}_MEAN"]),label=cname0)
#plt.plot(x,nresM,label='mobile O')
#plt.plot(x,nresF,label='detached O')
#plt.plot(x,par[5:]/np.sum(par[5:]),label='par')
plt.ylabel('Value')
plt.xlabel('Region number')
# show a legend on the plot
plt.legend()
plt.title("Utarg vs par") 
# function to show the plot
plt.show()


In [ ]:
x=np.array(range(expD.shape[0]),dtype=int)
plt.scatter(utarg,dv,label='uM')
#plt.plot(x,nresM,label='mobile O')
#plt.plot(x,nresF,label='detached O')
#plt.plot(x,par[5:]/np.sum(par[5:]),label='par')
plt.ylabel('dv')
plt.xlabel('utarg')
# show a legend on the plot
plt.legend()
plt.title("Utarg vs par") 
# function to show the plot
plt.show()


In [ ]:
x=np.array(range(expD.shape[0]),dtype=int)
fig, ax = plt.subplots()
ax.plot(x,target,'o-',label=cname7)
#plt.plot(x,nresM,label='mobile O')
#plt.plot(x,nresF,label='detached O')
ax.plot(x,resM,label='mobile DC')
ax.plot(x,resF,label='detached DC')
ax.set_xlabel('Region number')
ax.set_ylabel('Value')
ax.set_xticks(x,expD['Abbreviation'],rotation=90)
# show a legend on the plot
ax.legend()
pos = ax.get_position()
ax.set_position([pos.x0, pos.y0 + pos.height*0.15, pos.width , pos.height* 0.85])
plt.title(f"Cost = {format(np.sum((resF/(1-mProp) - target) ** 2),'.4g')}, $\chi^2$ = {format(np.sum(((resF/(1-mProp) - target)/targSD) ** 2),'.4g')}") 
# function to show the plot
plt.show()
#plt.savefig('bestFitCA1_20reg_1dv.png')
#plt.savefig('bestFitCA1_20reg_1dv.pdf')
#plt.savefig('bestFitCA1_20reg_1dv.svg')


In [ ]:
x=np.array(range(expD.shape[0]),dtype=int)
fig, ax = plt.subplots()
#ax.scatter(x,target,label=cname7)
ax.plot(x,target,'o-',label=cname7)
#plt.plot(x,nresM,label='mobile O')
#plt.plot(x,nresF,label='detached O')
#ax.plot(x,resM,label='mobile DC')
#ax.plot(x,resF/(1-mProp),linestyle=':', linewidth=5,label='detached norm DC')
ax.plot(x,resF/(1-mProp),label='detached norm DC')
ax.set_xlabel('Region number')
ax.set_ylabel('Value')
ax.set_xticks(x,expD['Abbreviation'],rotation=90)
# show a legend on the plot
ax.legend()
pos = ax.get_position()
ax.set_position([pos.x0, pos.y0 + pos.height*0.15, pos.width , pos.height* 0.85])
plt.title(f"Cost = {format(np.sum((resF/(1-mProp) - target) ** 2),'.4g')}, $\chi^2$ = {format(np.sum(((resF/(1-mProp) - target)/targSD) ** 2),'.4g')}") 
# function to show the plot
plt.show()
#plt.savefig('bestFitCA1_20reg_1dv_norm202406161015best.png')
#plt.savefig('bestFitCA1_20reg_1dv_norm202406161015best.pdf')
#plt.savefig('bestFitCA1_20reg_1dv_norm202406161015best.svg')


In [ ]:
fig, ax = plt.subplots()
ax.errorbar(x,target,targSD,label=cname7, fmt='-o')
ax.plot(x,resF,label='detached DC')
ax.plot(x,resF/(1-mProp),label='detached norm DC')
ax.set_xlabel('Region number')
ax.set_ylabel('Value')
ax.set_xticks(x,expD['Abbreviation'],rotation=90)
# show a legend on the plot
ax.legend()
#plt.show()
pos = ax.get_position()
ax.set_position([pos.x0, pos.y0 + pos.height*0.15, pos.width , pos.height* 0.85])
#plt.plot(x,resM+resF,label='total DC')
plt.title(f"Cost = {format(np.sum((resF/(1-mProp) - target) ** 2),'.4g')}, $\chi^2$ = {format(np.sum(((resF/(1-mProp) - target)/targSD) ** 2),'.4g')}") 
plt.show()
#plt.savefig('bestFitCA1_20reg_1dv_resF202406161015.png')
#plt.savefig('bestFitCA1_20reg_1dv_resF202406161015.pdf')
#plt.savefig('bestFitCA1_20reg_1dv_resF202406161015.svg')


## Apparent degradation rate

In [ ]:
nframes = 50
utrace = calcUtrace(par,nframes=50)
t = np.logspace(-0.5,math.log10(day7),nframes)
(t,7*t/day7,utrace.shape,t.shape,utrace[:3,:])

In [ ]:
resList=[]
for i in range(nframes):
    resm, resf = sushibelt.aggregate_segments(utrace[:, i+1], segIdx, expD['Abbreviation'], fun=np.sum)
    resList.append(resf)
resfree = np.array(resList)
(resF.shape,resfree.shape)

In [ ]:
dvA = par[3]
dvB = par[4]
dv = (10 ** dvA) + (10 ** dvB)*np.array(par[5:])
(expD.shape,expD['Abbreviation'],par[5:],dv,np.log(2)*7/(np.array(dv * day7)))

In [ ]:
i = 1
bfit, afit = np.polyfit(np.log(t), np.log(resfree[:,i]), deg=1)
plt.scatter(np.log(t), np.log(resfree[:,i]))
plt.plot(np.log(t), afit + bfit * np.log(t), color="k", lw=2.5)
(bfit, afit,f'LR equation: $Y = {round(afit*1e6,3)}'+'\\ 10^{-6}'+f' + {round(bfit*1e6,3)}'+'\\ 10^{-6}\\ X$')

In [ ]:
alpha = (np.log(resfree[0,i])-np.log(resfree[-1,i]))/day7
t_half = np.log(2)*7/(alpha * day7)
(alpha, t_half,format(alpha, ".2g"))

In [ ]:
x = 7*t/day7
alpha = np.zeros(expD.shape[0])
for i in range(expD.shape[0]):
    alpha[i] = (np.log(resfree[0,i])-np.log(resfree[-1,i]))/day7
    t_half = np.log(2)*7/(alpha[i] * day7)
    plt.plot(x,resfree[:,i],label=f"{expD['Abbreviation'][i]}")
    plt.scatter(x,resfree[:,i])
    plt.ylabel('Value')
    plt.xlabel('Time (days)')
    # show a legend on the plot
    plt.legend()
    plt.title(f"{expD['Abbreviation'][i]}: $\\alpha$ = {format(alpha[i], '.2g')}, T = {format(t_half, '.2g')} days.") 
    plt.show()
    #plt.savefig(f"CA1_20reg_1dv_resF_{expD['Abbreviation'][i]}.png")
    #plt.savefig(f"CA1_20reg_1dv_resF_{expD['Abbreviation'][i]}.pdf")
    #plt.savefig(f"CA1_20reg_1dv_resF_{expD['Abbreviation'][i]}.svg")
    plt.close()



In [ ]:
x = 7*t/day7
alpha = np.zeros(expD.shape[0])
fig, ax = plt.subplots()
for i in range(expD.shape[0]):
    alpha[i] = (np.log(resfree[0,i])-np.log(resfree[-1,i]))/day7
    t_half = np.log(2)*7/(alpha[i] * day7)
    ax.plot(x,resfree[:,i],label=f"{expD['Abbreviation'][i]}")
    ax.scatter(x,resfree[:,i])
ax.set_ylabel('Value')
ax.set_xlabel('Time (days)')
pos = ax.get_position()
ax.set_position([pos.x0, pos.y0, pos.width * 0.85, pos.height])
ax.legend(loc='center right', bbox_to_anchor=(1.3, 0.5))
# show a legend on the plot
plt.show()
#plt.savefig(f"CA1_20reg_1dv_resF.png")
#plt.savefig(f"CA1_20reg_1dv_resF.pdf")
#plt.savefig(f"CA1_20reg_1dv_resF.svg")



In [ ]:
x = 7*t/day7
alpha = np.zeros(expD.shape[0])
fig, axes = plt.subplots(5, 4, figsize=(16, 20))

k = 0
for i in range(5):
    for j in range(4):
        alpha[k] = (np.log(resfree[0,k])-np.log(resfree[-1,k]))/day7
        t_half = np.log(2)*7/(alpha[k] * day7)
        axes[i,j].plot(x,resfree[:,k],label=f"{expD['Abbreviation'][k]}")
        axes[i,j].scatter(x,resfree[:,k])
        axes[i,j].legend()
        if i == 4 :
            axes[i,j].set_xlabel('Time (days)')
        if j == 0 :
            axes[i,j].set_ylabel('Value')
        k += 1
#plt.ylabel('Value')
#plt.xlabel('Time (days)')
# show a legend on the plot
#plt.legend(bbox_to_anchor = (1.05,1.04),loc='upper left')
plt.show()
#plt.savefig(f"CA1_20reg_1dv_resF_panels.png")
#plt.savefig(f"CA1_20reg_1dv_resF_panels.pdf")
#plt.savefig(f"CA1_20reg_1dv_resF_panels.svg")



In [ ]:
x=np.array(range(expD.shape[0]),dtype=int)
fig, axes = plt.subplots(2, 1, figsize=(8, 10))

axes[0].plot(x,np.log(2)*7/(alpha * day7),label='half-life (days)')
#axes[0].xlabel('Region number')
#axes[0].ylabel('Apparent half-life (days)')
# show a legend on the plot
axes[0].legend()
plt.xticks(x,expD['Abbreviation'],rotation=90)
axes[1].plot(x,par[5:],label=f'Demand')
#axes[1].xlabel('Region number')
#axes[1].ylabel('Demand')
# show a legend on the plot
axes[1].legend()
plt.show()
#plt.savefig('CA1_20reg_1dv_A_demand202406161015.png')
#plt.savefig('CA1_20reg_1dv_A_demand202406161015.pdf')
#plt.savefig('CA1_20reg_1dv_A_demand202406161015.svg')


In [ ]:
x=np.array(range(expD.shape[0]),dtype=int)
uS, uS = sushibelt.aggregate_segments(np.concatenate((utarg,utarg)), segIdx, expD['Abbreviation'], fun=np.sum)
#fig, axes = plt.subplots(2, 1, figsize=(8, 10))

plt.scatter(uS,np.log(2)*7/(alpha * day7),label='half-life (days)')
plt.show()
#plt.savefig('CA1_20reg_1dv_A_demand202406161015.png')
#plt.savefig('CA1_20reg_1dv_A_demand202406161015.pdf')
#plt.savefig('CA1_20reg_1dv_A_demand202406161015.svg')


In [ ]:
A = (np.log(expD[f"{cname0}_MEAN"])-np.log(expD[f"{cname7}_MEAN"]))/day7
fig, ax = plt.subplots()
ax.plot(x,np.log(2)*7/(A * day7),label='half-life (days)')
ax.set_xlabel('Region number')
ax.set_ylabel('Apparent half-life (days)')
ax.set_xticks(x,expD['Abbreviation'],rotation=90)
# show a legend on the plot
ax.legend()
pos = ax.get_position()
ax.set_position([pos.x0, pos.y0 + pos.height*0.15, pos.width , pos.height* 0.85])
plt.show()
#plt.savefig('CA1_20reg_1dv_A.png')
#plt.savefig('CA1_20reg_1dv_A.pdf')
#plt.savefig('CA1_20reg_1dv_A.svg')


In [ ]:
A = (np.log(expD[f"{cname0}_MEAN"])-np.log(expD[f"{cname7}_MEAN"]))/day7
dvB = par[4]
dv = (10 ** dvA) + (10 ** dvB)*np.array(par[5:])
fig, ax = plt.subplots()
ax.plot(x,np.log(2)*7/(A * day7),label='Apparent half-life (days)')
ax.plot(x,np.log(2)*7/(dv * day7),label='Model half-life (days)')
ax.set_xlabel('Region number')
ax.set_ylabel('half-life (days)')
ax.set_xticks(x,expD['Abbreviation'],rotation=90)
# show a legend on the plot
ax.legend()
pos = ax.get_position()
ax.set_position([pos.x0, pos.y0 + pos.height*0.15, pos.width , pos.height* 0.85])
plt.show()
#plt.savefig('CA1_20reg_1dv_twoHalfLifes.png')
#plt.savefig('CA1_20reg_1dv_twoHalfLifes.pdf')
#plt.savefig('CA1_20reg_1dv_twoHalfLifes.svg')


In [ ]:
fig, ax = plt.subplots()
ax.plot(x,par[5:],label=f'Demand')
ax.set_xlabel('Region number')
ax.set_ylabel('Value')
ax.set_xticks(x,expD['Abbreviation'],rotation=90)
# show a legend on the plot
ax.legend()
pos = ax.get_position()
ax.set_position([pos.x0, pos.y0 + pos.height*0.15, pos.width , pos.height* 0.85])
plt.show()
#plt.savefig('CA1_20reg_1dv_demand202406161015.png')
#plt.savefig('CA1_20reg_1dv_demand202406161015.pdf')
#plt.savefig('CA1_20reg_1dv_demand202406161015.svg')


In [ ]:
COLOR_alpha = "#69b3a2"
COLOR_demand = "#3399e6"
fig, ax1 = plt.subplots(figsize=(8, 6))
# Instantiate a second axes that shares the same x-axis
ax2 = ax1.twinx()  
ax2.set_ylim(0, 1)
ax1.plot(x,np.log(2)*7/(A * day7),color = COLOR_alpha)
ax1.set_ylabel('Apparent half-life (days)',color = COLOR_alpha)
ax2.plot(x,par[5:],color = COLOR_demand)
ax2.set_ylabel('Demand',color = COLOR_demand)
ax1.set_xticks(x,expD['Abbreviation'],rotation=90)
pos = ax1.get_position()
ax1.set_position([pos.x0, pos.y0 + pos.height*0.15, pos.width , pos.height* 0.85])
#pos = ax2.get_position()
#ax2.set_position([pos.x0, pos.y0 + pos.height*0.15, pos.width , pos.height* 0.85])
plt.show()
#plt.savefig('CA1_20reg_1dv_Ademand202406161015.png')
#plt.savefig('CA1_20reg_1dv_Ademand202406161015.pdf')
#plt.savefig('CA1_20reg_1dv_Ademand202406161015.svg')


In [ ]:
bfit, afit = np.polyfit(par[5:],np.log(2)*7/(A * day7), deg=1)
xseq = np.linspace(min(par[5:]), max(par[5:]), num=100)
plt.scatter(par[5:],np.log(2)*7/(A * day7))
plt.plot(xseq, afit + bfit * xseq, color="k", lw=2.5)
plt.text(0, 13, f'LR equation: $Y = {round(afit,3)} + {round(bfit,3)} X$', fontsize=10)
plt.xlabel('Demand')
plt.ylabel('Apparent half-life (days)')
# show a legend on the plot
plt.show()
#plt.savefig('CA1_20reg_1dv_T_vs_demand202406161015.png')
#plt.savefig('CA1_20reg_1dv_T_vs_demand202406161015.pdf')
#plt.savefig('CA1_20reg_1dv_T_vs_demand202406161015.svg')


In [ ]:
delta=bgSignal
F = par[0]
Ctau = 10 ** par[1]
mProp = par[2]
dvA = par[3]
dvB = par[4]
dv = np.zeros(N)
utarg = delta*np.ones(N)
for k in range(N):
    if itarg[k] > 4:
        utarg[k] = par[itarg[k]]
        dv[k] = (10 ** dvA) + (10 ** dvB)*utarg[k]
utarg /= np.sum(utarg)
K = np.sum(utarg) / N
x = trafficking_solution(F * utarg + (1 - F) * K)
a = (1 / (1 + x))
aM, aF = sushibelt.aggregate_segments(np.concatenate((list(a),list(a))), segIdx, expD['Abbreviation'], fun=np.mean)
b = list((1 / (1 + x ** -1)))
bM, bF = sushibelt.aggregate_segments(np.concatenate((b,b)), segIdx, expD['Abbreviation'], fun=np.mean)
l = list(dv)
dM, dF = sushibelt.aggregate_segments(np.concatenate((l,l)), segIdx, expD['Abbreviation'], fun=np.mean)
c = list(Ctau * utarg / (F * utarg + (1 - F) * K))
cM, cF = sushibelt.aggregate_segments(np.concatenate((c,c)), segIdx, expD['Abbreviation'], fun=np.mean)
uM, uF = sushibelt.aggregate_segments(np.concatenate((utarg,utarg)), segIdx, expD['Abbreviation'], fun=np.mean)
df=pd.DataFrame([aM,bM,cM,dM,uM,par[5:],A,np.log(2)*7/(A * day7)],columns=expD['Abbreviation'],index=['a','b','c','d','utarg','demand','A','aT1/2'])
#df.to_csv('CA1_20reg_1dv_experiment.mean.heatmap.csv')
df

Day 7 3 weeks
----

In [ ]:
bgSignal = 1e-5
cname0='D0W3'
d0w = -1 * np.ones(N)
for i in range(expD.shape[0]):
    abb = expD['Abbreviation'][i]
    sidx= segIdx[abb]
    d0w[sidx] *= -1*expD[f"{cname0}_MEAN"][i]/len(sidx)
for i in range(N):
    if d0w[i]<0:
        d0w[i] = bgSignal
dinit = d0w/np.sum(d0w)

cname7='D7W3'
d7w = -1*np.ones(N)
for i in range(expD.shape[0]):
    abb = expD['Abbreviation'][i]
    sidx= segIdx[abb]
    d7w[sidx] *= -1 * expD[f"{cname7}_MEAN"][i]/len(sidx)
for i in range(N):
    if d7w[i]<0:
        d7w[i] = bgSignal
target = np.array(expD[f"{cname7}_MEAN"])/np.sum(expD[f"{cname0}_MEAN"]) #norm target to Day0 sum to take into accound degradation
targSD = np.array(expD[f"{cname7}_SD"])/np.sum(expD[f"{cname0}_MEAN"]) #measurement errors
tnorm = np.sum(target ** 2)
day7 = 7 * 24 * 3600 # final time point

itarg = np.ones(N, dtype=int)
for i in range(expD.shape[0]):
    abb = expD['Abbreviation'][i]
    sidx = segIdx[abb]
    itarg[sidx] *= (i+5)

#par = [0.111784701917933,-1,0.584647952749801,-5.40926182202759,-7.58373647291078,0.93075347720533,0.150119155946245,0.250026616923097,0.247923892623914,1,0.829698078092076,0.799680386796054,0.435204401984173,0.776990953492751,0.658564397472255,0.669466722069782,0.13217494651507,0.452337127435389,0.818398235362628,0.287266237055001,0.693374804035749,0.387600533946115,0.860396244028168,0.793857689742176,0.00234423876609767]
#cost=0.2289827
#par = [0.141527769926587,-1.04440055844237,0.381706990604462,-5.43766625150405,-12.234744035647,1,0.164098437336494,0.272405308568442,0.427804635567134,0.812746641751752,0.919149761897559,0.672064846552212,0.149965791068772,0.539839422213906,0.812188937409397,0.806685239524521,0.165041822978698,0.529423937850048,0.828384035574742,0.221525313745652,0.676304529167494,0.634182580029409,0.814899037032692,0.790009814378913,0.0884319910464033]
#cost=0.1914477
#par = [1.0,-18.0,0.339665263904274,-18.0,-0.1056091212860899,1.0,0.06274608035463802,4.4574543322602384e-07,0.02193672327975435,0.0870196802604857,8.738149616920519e-07,1.0,0.23641992030781905,0.20499095398769174,3.3139951083305927e-07,0.0264354594647535,1.0601425384202188e-07,0.018642526550405554,1.3229426155908984e-07,0.321659768251798,1.256853193752937e-06,0.9999991575763989,0.9008918582308576,1.0002936595814933e-07,1.0]
#cost = 0.00972634
#par = [0.18124010722657255,-18.0,0.4770849437408547,-18.0,-7.528208406834074,0.6335729107600226,0.28276999275111847,0.558910113544302,1.0,0.3929830308454687,0.2992123958197709,0.753091076326252,1.0,0.17238067898531845,1.0,1e-07,1.0,0.04290036974800929,0.9692956552909402,1.0,0.5346212757781438,1.0,0.28625682786833306,1e-07,1e-07]
#cost = 0.02659
#par = [0.32023059450279123,-6.327308855004608,0.627385538130974,-18.0,-17.999431588729976,0.06854654347167044,0.060486839296889444,0.04484900714349323,0.03490854158366522,0.00024252729610862994,0.32943753444660906,0.4029203029951907,0.12801603593190886,0.14871930210853104,0.14171491277653797,0.18024101228415457,0.18628112282529372,0.23357327869041905,0.4302044118823936,0.5008667022509936,0.0038157043176590766,0.2007104314288199,0.22494314014617123,0.31189759389623783,0.9999999999990268]
#cost = 0.001142
#par = [0.01385481065982917,-6.226265838840085,0.6709771115511489,-17.995913305482574,-16.696101588995024,0.21868548101204954,0.06211405206769383,0.048882353452234434,0.046632773486508856,1.9401449712242143e-06,0.7864443640945032,0.5632548318074286,0.14915702983161863,0.1491589398028194,0.13322800132469398,0.15668388533325328,0.17435550169553826,0.20436446923690443,0.37541096611836217,0.5260391371129066,0.043995168941356404,0.1970021740399707,0.19622524557390722,0.2758337024352678,0.9999999999613025]
#cost = 1.51e-09
par = [0.8861278052764772,-5.358123919558814,0.3127455791698551,-5.7356821635860165,-17.999032750284783,0.21382364663804718,0.06074280874523973,0.04781382417616123,0.04562340830020193,1.0000000000039887e-07,0.7733439929081787,0.5543045057407645,0.1468913931288651,0.14710118884157364,0.1316203146580693,0.15499357159791044,0.17296720163305693,0.20287137824434023,0.3735878952746845,0.5243019440900127,0.043864102155039865,0.1966480252602025,0.19591970216660096,0.2755074889018492,0.9999999999999992]
cost = 3.331839554992909e-11
mProp = par[2]
utrace = calcUtrace(par)
resM, resF = sushibelt.aggregate_segments(utrace[:, -1], segIdx, expD['Abbreviation'], fun=np.sum)


In [ ]:
{'F' : par[0],
'Ctau' : 10 ** par[1],
'mProp' : par[2],
'dvA': 10 ** par[3],
'dvB': 10 ** par[4],
'T1/2': np.log(2)*7/((10 ** par[3]) * day7),
'demand_CA1so_1' : par[5],
'demand_CA1so_2' : par[6],
'demand_CA1so_3' : par[7],
'demand_CA1so_4' : par[8],
'demand_CA1so_5' : par[9],
'demand_CA1sr_1' : par[10],
'demand_CA1sr_2' : par[11],
'demand_CA1sr_3' : par[12],
'demand_CA1sr_4' : par[13],
'demand_CA1sr_5' : par[14],
'demand_CA1sr_6' : par[15],
'demand_CA1sr_7' : par[16],
'demand_CA1sr_8' : par[17],
'demand_CA1sr_9' : par[18],
'demand_CA1sr_10' : par[19],
'demand_CA1slm_1' : par[20],
'demand_CA1slm_2' : par[21],
'demand_CA1slm_3' : par[22],
'demand_CA1slm_4' : par[23],
'demand_CA1slm_5' : par[24],
'sum_nobsDnn' : np.sum(nobsDnn),
'sum_nobsDnn^2' : np.sum(nobsDnn ** 2),
'sum_resM' : np.sum(resM),
'sum_resF' : np.sum(resF),
'sum_tot' : np.sum(resF)+np.sum(resM),
'err' : np.sum((resF/(1-mProp) - target) ** 2),
'err_norm' : np.sum((resF/(1-mProp) - target) ** 2)/tnorm,
'chi2' : np.sum(((resF/(1-mProp) - target)/targSD) ** 2),
'cost' : cost
}

In [ ]:
x=np.array(range(expD.shape[0]),dtype=int)
fig, ax = plt.subplots()
ax.plot(x,target,'o-',label=cname7)
#plt.plot(x,nresM,label='mobile O')
#plt.plot(x,nresF,label='detached O')
ax.plot(x,resM,label='mobile DC')
ax.plot(x,resF,label='detached DC')
ax.set_xlabel('Region number')
ax.set_ylabel('Value')
ax.set_xticks(x,expD['Abbreviation'],rotation=90)
# show a legend on the plot
ax.legend(loc='upper left')
pos = ax.get_position()
ax.set_position([pos.x0, pos.y0 + pos.height*0.15, pos.width , pos.height* 0.85])
plt.title(f"Cost = {format(np.sum((resF/(1-mProp) - target) ** 2),'.4g')}, $\chi^2$ = {format(np.sum(((resF/(1-mProp) - target)/targSD) ** 2),'.4g')}") 
# function to show the plot
plt.show()
#plt.savefig('bestFitCA1_20reg_1dv_3w.png')
#plt.savefig('bestFitCA1_20reg_1dv_3w.pdf')
#plt.savefig('bestFitCA1_20reg_1dv_3w.svg')


In [ ]:
fig, ax = plt.subplots()
ax.errorbar(x,target,targSD,label=cname7, fmt='-o')
#ax.plot(x,resF,label='detached DC')
ax.plot(x,resF/(1-mProp),label='detached norm DC')
ax.set_xlabel('Region number')
ax.set_ylabel('Value')
ax.set_xticks(x,expD['Abbreviation'],rotation=90)
# show a legend on the plot
ax.legend()
pos = ax.get_position()
ax.set_position([pos.x0, pos.y0 + pos.height*0.15, pos.width , pos.height* 0.85])
plt.title(f"Cost = {format(np.sum((resF/(1-mProp) - target) ** 2),'.4g')}, $\chi^2$ = {format(np.sum(((resF/(1-mProp) - target)/targSD) ** 2),'.4g')}") 
# function to show the plot
plt.show()
#plt.savefig('bestFitCA1_20reg_1dv_3w_de.png')
#plt.savefig('bestFitCA1_20reg_1dv_3w_de.pdf')
#plt.savefig('bestFitCA1_20reg_1dv_3w_de.svg')


In [ ]:
def calcUtrace3par(par,delta=bgSignal):
    F = bpar[0]
    Ctau = 10 ** par[0]
    mProp = bpar[2]
    dvA = par[1]
    dvB = par[2]
    dv = np.zeros(N)
    utarg = delta*np.ones(N)
    for k in range(N):
        if itarg[k] > 4:
            utarg[k] = bpar[itarg[k]]
            dv[k] = (10 ** dvA) + (10 ** dvB)*utarg[k]
    utarg /= np.sum(utarg)
    K = np.sum(utarg) / N
    x = trafficking_solution(F * utarg + (1 - F) * K)
    a = (1 / (1 + x))
    a = list(a)
    b = list((1 / (1 + x ** -1)))
    l = list(dv)
    c = list(Ctau * utarg / (F * utarg + (1 - F) * K))
    d = list(np.zeros(N))
    A = sushi_system(a, b, c, d, l)
    u0 = np.concatenate((mProp * dinit, (1 - mProp) * dinit))
    utrace = sim_time(A, u0, day7)
    return utrace

#par = [-16.81120695082292,-5.304089695904444,-12.387273346439557]
#cost = 3.6963521403227104
#par = [-6.146053650006426,-5.284385195872606,-18.0]
#cost = 1.4121248688210135
#par = [-5.613771283822604,-5.2215439090716185,-18.0]
#cost = 0.9900881157516854
par = [-5.591517353999318,-5.266710638487275,-17.999895594052166]
cost = 4.5368536883995205e-05
mProp = bpar[2]
utrace = calcUtrace3par(par)
resM, resF = sushibelt.aggregate_segments(utrace[:, -1], segIdx, expD['Abbreviation'], fun=np.sum)
resF_3par = resF

In [ ]:
{'Ctau' : 10 ** par[0],
'dvA': 10 ** par[1],
'dvB': 10 ** par[2],
'T1/2': np.log(2)*7/((10 ** par[1]) * day7),
'sum_nobsDnn' : np.sum(nobsDnn),
'sum_nobsDnn^2' : np.sum(nobsDnn ** 2),
'sum_resM' : np.sum(resM),
'sum_resF' : np.sum(resF),
'sum_tot' : np.sum(resF)+np.sum(resM),
'err' : np.sum((resF/(1-mProp) - target) ** 2),
'err_norm' : np.sum((resF/(1-mProp) - target) ** 2)/tnorm,
'chi2' : np.sum(((resF/(1-mProp) - target)/targSD) ** 2),
'cost' : cost
}

In [ ]:
dvA = par[1]
dvB = par[2]
dv = (10 ** dvA) + (10 ** dvB)*np.array(bpar[5:])
(expD.shape,expD['Abbreviation'],bpar[5:],dv,np.log(2)*7/(np.array(dv * day7)))

In [ ]:
x=np.array(range(expD.shape[0]),dtype=int)
fig, ax = plt.subplots()

ax.plot(x,target,'o-',label=cname7)
#plt.plot(x,nresM,label='mobile O')
#plt.plot(x,nresF,label='detached O')
#ax.plot(x,resM,label='mobile DC')
ax.plot(x,resF,label='detached DC')
ax.set_xlabel('Region number')
ax.set_ylabel('Value')
ax.set_xticks(x,expD['Abbreviation'],rotation=90)
# show a legend on the plot
ax.legend()
pos = ax.get_position()
ax.set_position([pos.x0, pos.y0 + pos.height*0.15, pos.width , pos.height* 0.85])
plt.title(f"Cost = {format(np.sum((resF/(1-mProp) - target) ** 2),'.4g')}, $\chi^2$ = {format(np.sum(((resF/(1-mProp) - target)/targSD) ** 2),'.4g')}") 
# function to show the plot
#plt.show()
plt.savefig('bestFitCA1_20reg_1dv_fit3w_3par.png')
plt.savefig('bestFitCA1_20reg_1dv_fit3w_3par.pdf')
plt.savefig('bestFitCA1_20reg_1dv_fit3w_3par.svg')


In [ ]:
fig, ax = plt.subplots()
ax.plot(x,target,'o-',label=cname7)
#plt.errorbar(x,target,targSD,label=cname7, fmt='-o')
#ax.plot(x,resF,label='detached DC')
ax.plot(x,resF/(1-mProp),label='detached norm DC')
ax.set_xlabel('Region number')
ax.set_ylabel('Value')
ax.set_xticks(x,expD['Abbreviation'],rotation=90)
# show a legend on the plot
ax.legend()
pos = ax.get_position()
ax.set_position([pos.x0, pos.y0 + pos.height*0.15, pos.width , pos.height* 0.85])
plt.title(f"Cost = {format(np.sum((resF/(1-mProp) - target) ** 2),'.4g')}, $\chi^2$ = {format(np.sum(((resF/(1-mProp) - target)/targSD) ** 2),'.4g')}") 
# function to show the plot
plt.show()
#plt.savefig('bestFitCA1_20reg_1dv_fit3w_3par_de.png')
#plt.savefig('bestFitCA1_20reg_1dv_fit3w_3par_de.pdf')
#plt.savefig('bestFitCA1_20reg_1dv_fit3w_3par_de.svg')


In [ ]:
fig, ax = plt.subplots()
#plt.plot(x,target,label=cname7)
ax.errorbar(x,target,targSD,label=cname7, fmt='-o')
#plt.plot(x,resF,label='detached DC')
ax.plot(x,resF/(1-mProp),label='detached norm DC')
ax.set_xlabel('Region number')
ax.set_ylabel('Value')
ax.set_xticks(x,expD['Abbreviation'],rotation=90)
# show a legend on the plot
ax.legend(loc='upper left')
pos = ax.get_position()
ax.set_position([pos.x0, pos.y0 + pos.height*0.15, pos.width , pos.height* 0.85])
plt.title(f"Cost = {format(np.sum((resF/(1-mProp) - target) ** 2),'.4g')}, $\chi^2$ = {format(np.sum(((resF/(1-mProp) - target)/targSD) ** 2),'.4g')}") 
# function to show the plot
plt.show()
#plt.savefig('bestFitCA1_20reg_1dv_fit3w_3par_deE.png')
#plt.savefig('bestFitCA1_20reg_1dv_fit3w_3par_deE.pdf')
#plt.savefig('bestFitCA1_20reg_1dv_fit3w_3par_deE.svg')


In [ ]:
def calcUtrace2par(par,delta=bgSignal):
    F = bpar[0]
    Ctau = 10 ** bpar[1]
    mProp = bpar[2]
    dvA = par[0]
    dvB = par[1]
    dv = np.zeros(N)
    utarg = delta*np.ones(N)
    for k in range(N):
        if itarg[k] > 4:
            utarg[k] = bpar[itarg[k]]
            dv[k] = (10 ** dvA) + (10 ** dvB)*utarg[k]
    utarg /= np.sum(utarg)
    K = np.sum(utarg) / N
    x = trafficking_solution(F * utarg + (1 - F) * K)
    a = (1 / (1 + x))
    a = list(a)
    b = list((1 / (1 + x ** -1)))
    l = list(dv)
    c = list(Ctau * utarg / (F * utarg + (1 - F) * K))
    d = list(np.zeros(N))
    A = sushi_system(a, b, c, d, l)
    u0 = np.concatenate((mProp * dinit, (1 - mProp) * dinit))
    utrace = sim_time(A, u0, day7)
    return utrace

par = [-5.283019429412654,-18.0]
cost = 4.872024346150998e-05
mProp = bpar[2]
utrace = calcUtrace2par(par)
resM, resF = sushibelt.aggregate_segments(utrace[:, -1], segIdx, expD['Abbreviation'], fun=np.sum)
resF_2par = resF

In [ ]:
{'dvA': 10 ** par[0],
'dvB': 10 ** par[1],
'T1/2': np.log(2)*7/(10 ** par[0] * day7),
'sum_nobsDnn' : np.sum(nobsDnn),
'sum_nobsDnn^2' : np.sum(nobsDnn ** 2),
'sum_resM' : np.sum(resM),
'sum_resF' : np.sum(resF),
'sum_tot' : np.sum(resF)+np.sum(resM),
'err' : np.sum((resF/(1-mProp) - target) ** 2),
'err_norm' : np.sum((resF/(1-mProp) - target) ** 2)/tnorm,
'chi2' : np.sum(((resF/(1-mProp) - target)/targSD) ** 2),
'cost' : cost
}

In [ ]:
fig, ax = plt.subplots()
ax.plot(x,target,'o-',label=cname7)
#plt.errorbar(x,target,targSD,label=cname7, fmt='-o')
#ax.plot(x,resF,label='detached DC')
ax.plot(x,resF/(1-mProp),label='detached norm DC')
ax.set_xlabel('Region number')
ax.set_ylabel('Value')
ax.set_xticks(x,expD['Abbreviation'],rotation=90)
# show a legend on the plot
ax.legend()
pos = ax.get_position()
ax.set_position([pos.x0, pos.y0 + pos.height*0.15, pos.width , pos.height* 0.85])
plt.title(f"Cost = {format(np.sum((resF/(1-mProp) - target) ** 2),'.4g')}, $\chi^2$ = {format(np.sum(((resF/(1-mProp) - target)/targSD) ** 2),'.4g')}") 
# function to show the plot
plt.show()
#plt.savefig('bestFitCA1_20reg_1dv_fit3w_2par_de.png')
#plt.savefig('bestFitCA1_20reg_1dv_fit3w_2par_de.pdf')
#plt.savefig('bestFitCA1_20reg_1dv_fit3w_2par_de.svg')


In [ ]:
fig, ax = plt.subplots()
#plt.plot(x,target,label=cname7)
ax.errorbar(x,target,targSD,label=cname7, fmt='-o')
#plt.plot(x,resF,label='detached DC')
ax.plot(x,resF/(1-mProp),label='detached norm DC')
ax.set_xlabel('Region number')
ax.set_ylabel('Value')
ax.set_xticks(x,expD['Abbreviation'],rotation=90)
# show a legend on the plot
ax.legend(loc='upper left')
pos = ax.get_position()
ax.set_position([pos.x0, pos.y0 + pos.height*0.15, pos.width , pos.height* 0.85])
plt.title(f"Cost = {format(np.sum((resF/(1-mProp) - target) ** 2),'.4g')}, $\chi^2$ = {format(np.sum(((resF/(1-mProp) - target)/targSD) ** 2),'.4g')}") 
# function to show the plot
plt.show()
#plt.savefig('bestFitCA1_20reg_1dv_fit3w_2par_deE.png')
#plt.savefig('bestFitCA1_20reg_1dv_fit3w_2par_deE.pdf')
#plt.savefig('bestFitCA1_20reg_1dv_fit3w_2par_deE.svg')


In [ ]:
fig, ax = plt.subplots()
ax.plot(x,target,'o-',label=cname7)
#plt.errorbar(x,target,targSD,label=cname7, fmt='-o')
#ax.plot(x,resF,label='detached DC')
ax.plot(x,resF_3par/(1-mProp),label='3par')
ax.plot(x,resF_2par/(1-mProp),label='2par')
ax.set_xlabel('Region number')
ax.set_ylabel('Value')
ax.set_xticks(x,expD['Abbreviation'],rotation=90)
# show a legend on the plot
ax.legend()
pos = ax.get_position()
ax.set_position([pos.x0, pos.y0 + pos.height*0.15, pos.width , pos.height* 0.85])
plt.title(f"Compare 3par and 2par solutions") 
# function to show the plot
plt.show()


Day 0 18 month
----

In [ ]:
bgSignal = 1e-5
cname0='D0M318'
d0w = -1 * np.ones(N)
for i in range(expD.shape[0]):
    abb = expD['Abbreviation'][i]
    sidx= segIdx[abb]
    d0w[sidx] *= -1*expD[f"{cname0}_MEAN"][i]/len(sidx)
for i in range(N):
    if d0w[i]<0:
        d0w[i] = bgSignal
dinit = d0w/np.sum(d0w)

cname7='D7M318'
d7w = -1*np.ones(N)
for i in range(expD.shape[0]):
    abb = expD['Abbreviation'][i]
    sidx= segIdx[abb]
    d7w[sidx] *= -1 * expD[f"{cname7}_MEAN"][i]/len(sidx)
for i in range(N):
    if d7w[i]<0:
        d7w[i] = bgSignal
target = np.array(expD[f"{cname7}_MEAN"])/np.sum(expD[f"{cname0}_MEAN"]) #norm target to Day0 sum to take into accound degradation
targSD = np.array(expD[f"{cname7}_SD"])/np.sum(expD[f"{cname0}_MEAN"]) #measurement errors
tnorm = np.sum(target ** 2)
day7 = 7 * 24 * 3600 # final time point

itarg = np.ones(N, dtype=int)
for i in range(expD.shape[0]):
    abb = expD['Abbreviation'][i]
    sidx = segIdx[abb]
    itarg[sidx] *= (i+5)

#par = [0.111784701917933,-1,0.584647952749801,-5.40926182202759,-7.58373647291078,0.93075347720533,0.150119155946245,0.250026616923097,0.247923892623914,1,0.829698078092076,0.799680386796054,0.435204401984173,0.776990953492751,0.658564397472255,0.669466722069782,0.13217494651507,0.452337127435389,0.818398235362628,0.287266237055001,0.693374804035749,0.387600533946115,0.860396244028168,0.793857689742176,0.00234423876609767]
#cost=0.2289827
#par = [0.141527769926587,-1.04440055844237,0.381706990604462,-5.43766625150405,-12.234744035647,1,0.164098437336494,0.272405308568442,0.427804635567134,0.812746641751752,0.919149761897559,0.672064846552212,0.149965791068772,0.539839422213906,0.812188937409397,0.806685239524521,0.165041822978698,0.529423937850048,0.828384035574742,0.221525313745652,0.676304529167494,0.634182580029409,0.814899037032692,0.790009814378913,0.0884319910464033]
#cost=0.1914477
#par = [1.0,-18.0,0.339665263904274,-18.0,-0.1056091212860899,1.0,0.06274608035463802,4.4574543322602384e-07,0.02193672327975435,0.0870196802604857,8.738149616920519e-07,1.0,0.23641992030781905,0.20499095398769174,3.3139951083305927e-07,0.0264354594647535,1.0601425384202188e-07,0.018642526550405554,1.3229426155908984e-07,0.321659768251798,1.256853193752937e-06,0.9999991575763989,0.9008918582308576,1.0002936595814933e-07,1.0]
#cost = 0.00972634
#par = [0.18124010722657255,-18.0,0.4770849437408547,-18.0,-7.528208406834074,0.6335729107600226,0.28276999275111847,0.558910113544302,1.0,0.3929830308454687,0.2992123958197709,0.753091076326252,1.0,0.17238067898531845,1.0,1e-07,1.0,0.04290036974800929,0.9692956552909402,1.0,0.5346212757781438,1.0,0.28625682786833306,1e-07,1e-07]
#cost = 0.02659
#par = [0.32023059450279123,-6.327308855004608,0.627385538130974,-18.0,-17.999431588729976,0.06854654347167044,0.060486839296889444,0.04484900714349323,0.03490854158366522,0.00024252729610862994,0.32943753444660906,0.4029203029951907,0.12801603593190886,0.14871930210853104,0.14171491277653797,0.18024101228415457,0.18628112282529372,0.23357327869041905,0.4302044118823936,0.5008667022509936,0.0038157043176590766,0.2007104314288199,0.22494314014617123,0.31189759389623783,0.9999999999990268]
#cost = 0.001142
#par = [0.01385481065982917,-6.226265838840085,0.6709771115511489,-17.995913305482574,-16.696101588995024,0.21868548101204954,0.06211405206769383,0.048882353452234434,0.046632773486508856,1.9401449712242143e-06,0.7864443640945032,0.5632548318074286,0.14915702983161863,0.1491589398028194,0.13322800132469398,0.15668388533325328,0.17435550169553826,0.20436446923690443,0.37541096611836217,0.5260391371129066,0.043995168941356404,0.1970021740399707,0.19622524557390722,0.2758337024352678,0.9999999999613025]
#cost = 1.51e-09
par = [0.8861278052764772,-5.358123919558814,0.3127455791698551,-5.7356821635860165,-17.999032750284783,0.21382364663804718,0.06074280874523973,0.04781382417616123,0.04562340830020193,1.0000000000039887e-07,0.7733439929081787,0.5543045057407645,0.1468913931288651,0.14710118884157364,0.1316203146580693,0.15499357159791044,0.17296720163305693,0.20287137824434023,0.3735878952746845,0.5243019440900127,0.043864102155039865,0.1966480252602025,0.19591970216660096,0.2755074889018492,0.9999999999999992]
cost = 3.331839554992909e-11
mProp = par[2]
utrace = calcUtrace(par)
resM, resF = sushibelt.aggregate_segments(utrace[:, -1], segIdx, expD['Abbreviation'], fun=np.sum)


In [ ]:
{'F' : par[0],
'Ctau' : 10 ** par[1],
'mProp' : par[2],
'dvA': 10 ** par[3],
'dvB': 10 ** par[4],
'demand_CA1so_1' : par[5],
'demand_CA1so_2' : par[6],
'demand_CA1so_3' : par[7],
'demand_CA1so_4' : par[8],
'demand_CA1so_5' : par[9],
'demand_CA1sr_1' : par[10],
'demand_CA1sr_2' : par[11],
'demand_CA1sr_3' : par[12],
'demand_CA1sr_4' : par[13],
'demand_CA1sr_5' : par[14],
'demand_CA1sr_6' : par[15],
'demand_CA1sr_7' : par[16],
'demand_CA1sr_8' : par[17],
'demand_CA1sr_9' : par[18],
'demand_CA1sr_10' : par[19],
'demand_CA1slm_1' : par[20],
'demand_CA1slm_2' : par[21],
'demand_CA1slm_3' : par[22],
'demand_CA1slm_4' : par[23],
'demand_CA1slm_5' : par[24],
'sum_nobsDnn' : np.sum(nobsDnn),
'sum_nobsDnn^2' : np.sum(nobsDnn ** 2),
'sum_resM' : np.sum(resM),
'sum_resF' : np.sum(resF),
'sum_tot' : np.sum(resF)+np.sum(resM),
'err' : np.sum((resF/(1-mProp) - target) ** 2),
'err_norm' : np.sum((resF/(1-mProp) - target) ** 2)/tnorm,
'chi2' : np.sum(((resF/(1-mProp) - target)/targSD) ** 2),
'cost' : cost
}

In [ ]:
x=np.array(range(expD.shape[0]),dtype=int)
fig, ax = plt.subplots()

ax.plot(x,target,'o-',label='D7M18')
#plt.plot(x,nresM,label='mobile O')
#plt.plot(x,nresF,label='detached O')
ax.plot(x,resM,label='mobile DC')
ax.plot(x,resF,label='detached DC')
ax.set_xlabel('Region number')
ax.set_ylabel('Value')
ax.set_xticks(x,expD['Abbreviation'],rotation=90)
# show a legend on the plot
ax.legend()
pos = ax.get_position()
ax.set_position([pos.x0, pos.y0 + pos.height*0.15, pos.width , pos.height* 0.85])
plt.title(f"Cost = {format(np.sum((resF/(1-mProp) - target) ** 2),'.4g')}, $\chi^2$ = {format(np.sum(((resF/(1-mProp) - target)/targSD) ** 2),'.4g')}") 
# function to show the plot
plt.show()
#plt.savefig('bestFitCA1_20reg_1dv_18m.png')
#plt.savefig('bestFitCA1_20reg_1dv_18m.pdf')
#plt.savefig('bestFitCA1_20reg_1dv_18m.svg')


In [ ]:
fig, ax = plt.subplots()
ax.errorbar(x,target,targSD,label='D7M18', fmt='-o')
#ax.plot(x,resF,label='detached DC')
ax.plot(x,resF/(1-mProp),label='detached norm DC')
ax.set_xlabel('Region number')
ax.set_ylabel('Value')
ax.set_xticks(x,expD['Abbreviation'],rotation=90)
# show a legend on the plot
ax.legend()
pos = ax.get_position()
ax.set_position([pos.x0, pos.y0 + pos.height*0.15, pos.width , pos.height* 0.85])
plt.title(f"Cost = {format(np.sum((resF/(1-mProp) - target) ** 2),'.4g')}, $\chi^2$ = {format(np.sum(((resF/(1-mProp) - target)/targSD) ** 2),'.4g')}") 
# function to show the plot
plt.show()
#plt.savefig('bestFitCA1_20reg_1dv_18m_de.png')
#plt.savefig('bestFitCA1_20reg_1dv_18m_de.pdf')
#plt.savefig('bestFitCA1_20reg_1dv_18m_de.svg')


In [ ]:
def calcUtrace3par(par,delta=bgSignal):
    F = bpar[0]
    Ctau = 10 ** par[0]
    mProp = bpar[2]
    dvA = par[1]
    dvB = par[2]
    dv = np.zeros(N)
    utarg = delta*np.ones(N)
    for k in range(N):
        if itarg[k] > 4:
            utarg[k] = bpar[itarg[k]]
            dv[k] = (10 ** dvA) + (10 ** dvB)*utarg[k]
    utarg /= np.sum(utarg)
    K = np.sum(utarg) / N
    x = trafficking_solution(F * utarg + (1 - F) * K)
    a = (1 / (1 + x))
    a = list(a)
    b = list((1 / (1 + x ** -1)))
    l = list(dv)
    c = list(Ctau * utarg / (F * utarg + (1 - F) * K))
    d = list(np.zeros(N))
    A = sushi_system(a, b, c, d, l)
    u0 = np.concatenate((mProp * dinit, (1 - mProp) * dinit))
    utrace = sim_time(A, u0, day7)
    return utrace

#par = [-16.81120695082292,-5.304089695904444,-12.387273346439557]
#cost = 3.6963521403227104
#par = [-5.35361070901644,-5.781162290769705,-18.0]
#cost = 5.427795967245935
par = [-5.3473724799433935,-5.7875758356325395,-7.0200863650857634]
cost = 3.359950648864551e-05
mProp = bpar[2]
utrace = calcUtrace3par(par)
resM, resF = sushibelt.aggregate_segments(utrace[:, -1], segIdx, expD['Abbreviation'], fun=np.sum)
resF_3par = resF

In [ ]:
{'Ctau' : 10 ** par[0],
'dvA': 10 ** par[1],
'dvB': 10 ** par[2],
'sum_nobsDnn' : np.sum(nobsDnn),
'sum_nobsDnn^2' : np.sum(nobsDnn ** 2),
'sum_resM' : np.sum(resM),
'sum_resF' : np.sum(resF),
'sum_tot' : np.sum(resF)+np.sum(resM),
'err' : np.sum((resF/(1-mProp) - target) ** 2),
'err_norm' : np.sum((resF/(1-mProp) - target) ** 2)/tnorm,
'chi2' : np.sum(((resF/(1-mProp) - target)/targSD) ** 2),
'cost' : cost
}

In [ ]:
x=np.array(range(expD.shape[0]),dtype=int)
fig, ax = plt.subplots()

ax.plot(x,target,'o-',label='D7M18')
#plt.plot(x,nresM,label='mobile O')
#plt.plot(x,nresF,label='detached O')
ax.plot(x,resM,label='mobile DC')
ax.plot(x,resF,label='detached DC')
ax.set_xlabel('Region number')
ax.set_ylabel('Value')
ax.set_xticks(x,expD['Abbreviation'],rotation=90)
# show a legend on the plot
ax.legend()
pos = ax.get_position()
ax.set_position([pos.x0, pos.y0 + pos.height*0.15, pos.width , pos.height* 0.85])
plt.title(f"Cost = {format(np.sum((resF/(1-mProp) - target) ** 2),'.4g')}, $\chi^2$ = {format(np.sum(((resF/(1-mProp) - target)/targSD) ** 2),'.4g')}") 
# function to show the plot
plt.show()
#plt.savefig('bestFitCA1_20reg_1dv_fit18m_3par.png')
#plt.savefig('bestFitCA1_20reg_1dv_fit18m_3par.pdf')
#plt.savefig('bestFitCA1_20reg_1dv_fit18m_3par.svg')


In [ ]:
fig, ax = plt.subplots()
ax.plot(x,target,'o-',label='D7M18')
ax.plot(x,resF,label='detached DC')
ax.plot(x,resF/(1-mProp),label='detached norm DC')
ax.set_xlabel('Region number')
ax.set_ylabel('Value')
ax.set_xticks(x,expD['Abbreviation'],rotation=90)
# show a legend on the plot
ax.legend()
pos = ax.get_position()
ax.set_position([pos.x0, pos.y0 + pos.height*0.15, pos.width , pos.height* 0.85])
plt.title(f"Cost = {format(np.sum((resF/(1-mProp) - target) ** 2),'.4g')}, $\chi^2$ = {format(np.sum(((resF/(1-mProp) - target)/targSD) ** 2),'.4g')}") 
# function to show the plot
plt.show()
#plt.savefig('bestFitCA1_20reg_1dv_fit18m_3par_de.png')
#plt.savefig('bestFitCA1_20reg_1dv_fit18m_3par_de.pdf')
#plt.savefig('bestFitCA1_20reg_1dv_fit18m_3par_de.svg')


In [ ]:
fig, ax = plt.subplots()
ax.errorbar(x,target,targSD,label='D7M18', fmt='-o')
#ax.plot(x,resF,label='detached DC')
ax.plot(x,resF/(1-mProp),label='detached norm DC')
ax.set_xlabel('Region number')
ax.set_ylabel('Value')
ax.set_xticks(x,expD['Abbreviation'],rotation=90)
# show a legend on the plot
ax.legend()
pos = ax.get_position()
ax.set_position([pos.x0, pos.y0 + pos.height*0.15, pos.width , pos.height* 0.85])
plt.title(f"Cost = {format(np.sum((resF/(1-mProp) - target) ** 2),'.4g')}, $\chi^2$ = {format(np.sum(((resF/(1-mProp) - target)/targSD) ** 2),'.4g')}") 
# function to show the plot
plt.show()
#plt.savefig('bestFitCA1_20reg_1dv_fit18m_3par_de.png')
#plt.savefig('bestFitCA1_20reg_1dv_fit18m_3par_de.pdf')
#plt.savefig('bestFitCA1_20reg_1dv_fit18m_3par_de.svg')


In [ ]:
def calcUtrace2par(par,delta=bgSignal):
    F = bpar[0]
    Ctau = 10 ** bpar[1]
    mProp = bpar[2]
    dvA = par[0]
    dvB = par[1]
    dv = np.zeros(N)
    utarg = delta*np.ones(N)
    for k in range(N):
        if itarg[k] > 4:
            utarg[k] = bpar[itarg[k]]
            dv[k] = (10 ** dvA) + (10 ** dvB)*utarg[k]
    utarg /= np.sum(utarg)
    K = np.sum(utarg) / N
    x = trafficking_solution(F * utarg + (1 - F) * K)
    a = (1 / (1 + x))
    a = list(a)
    b = list((1 / (1 + x ** -1)))
    l = list(dv)
    c = list(Ctau * utarg / (F * utarg + (1 - F) * K))
    d = list(np.zeros(N))
    A = sushi_system(a, b, c, d, l)
    u0 = np.concatenate((mProp * dinit, (1 - mProp) * dinit))
    utrace = sim_time(A, u0, day7)
    return utrace

par = [-5.78767290580479,-7.018887371451555]
cost = 3.36090308526812e-05
mProp = bpar[2]
utrace = calcUtrace2par(par)
resM, resF = sushibelt.aggregate_segments(utrace[:, -1], segIdx, expD['Abbreviation'], fun=np.sum)
resF_2par = resF

In [ ]:
{'dvA': 10 ** par[0],
'dvB': 10 ** par[1],
'T1/2': np.log(2)*7/(10 ** par[0] * day7),
'sum_nobsDnn' : np.sum(nobsDnn),
'sum_nobsDnn^2' : np.sum(nobsDnn ** 2),
'sum_resM' : np.sum(resM),
'sum_resF' : np.sum(resF),
'sum_tot' : np.sum(resF)+np.sum(resM),
'err' : np.sum((resF/(1-mProp) - target) ** 2),
'err_norm' : np.sum((resF/(1-mProp) - target) ** 2)/tnorm,
'chi2' : np.sum(((resF/(1-mProp) - target)/targSD) ** 2),
'cost' : cost
}

In [ ]:
fig, ax = plt.subplots()
ax.plot(x,target,'o-',label=cname7)
#plt.errorbar(x,target,targSD,label=cname7, fmt='-o')
#ax.plot(x,resF,label='detached DC')
ax.plot(x,resF/(1-mProp),label='detached norm DC')
ax.set_xlabel('Region number')
ax.set_ylabel('Value')
ax.set_xticks(x,expD['Abbreviation'],rotation=90)
# show a legend on the plot
ax.legend()
pos = ax.get_position()
ax.set_position([pos.x0, pos.y0 + pos.height*0.15, pos.width , pos.height* 0.85])
plt.title(f"Cost = {format(np.sum((resF/(1-mProp) - target) ** 2),'.4g')}, $\chi^2$ = {format(np.sum(((resF/(1-mProp) - target)/targSD) ** 2),'.4g')}") 
# function to show the plot
plt.show()
#plt.savefig('bestFitCA1_20reg_1dv_fit18M_2par_de.png')
#plt.savefig('bestFitCA1_20reg_1dv_fit18M_2par_de.pdf')
#plt.savefig('bestFitCA1_20reg_1dv_fit18M_2par_de.svg')


In [ ]:
fig, ax = plt.subplots()
#plt.plot(x,target,label=cname7)
ax.errorbar(x,target,targSD,label=cname7, fmt='-o')
#plt.plot(x,resF,label='detached DC')
ax.plot(x,resF/(1-mProp),label='detached norm DC')
ax.set_xlabel('Region number')
ax.set_ylabel('Value')
ax.set_xticks(x,expD['Abbreviation'],rotation=90)
# show a legend on the plot
ax.legend(loc='upper left')
pos = ax.get_position()
ax.set_position([pos.x0, pos.y0 + pos.height*0.15, pos.width , pos.height* 0.85])
plt.title(f"Cost = {format(np.sum((resF/(1-mProp) - target) ** 2),'.4g')}, $\chi^2$ = {format(np.sum(((resF/(1-mProp) - target)/targSD) ** 2),'.4g')}") 
# function to show the plot
plt.show()
#plt.savefig('bestFitCA1_20reg_1dv_fit18M_2par_deE.png')
#plt.savefig('bestFitCA1_20reg_1dv_fit18M_2par_deE.pdf')
#plt.savefig('bestFitCA1_20reg_1dv_fit18M_2par_deE.svg')


In [ ]:
fig, ax = plt.subplots()
ax.plot(x,target,'o-',label=cname7)
#plt.errorbar(x,target,targSD,label=cname7, fmt='-o')
#ax.plot(x,resF,label='detached DC')
ax.plot(x,resF_3par/(1-mProp),label='3par')
ax.plot(x,resF_2par/(1-mProp),label='2par')
ax.set_xlabel('Region number')
ax.set_ylabel('Value')
ax.set_xticks(x,expD['Abbreviation'],rotation=90)
# show a legend on the plot
ax.legend()
pos = ax.get_position()
ax.set_position([pos.x0, pos.y0 + pos.height*0.15, pos.width , pos.height* 0.85])
plt.title(f"Compare 3par and 2par solutions") 
# function to show the plot
plt.show()


In [ ]:
dvA=np.array([1.8378828974092036e-06,5.4111473652332775e-06,1.6308881019486249e-06])
np.log(2)*7/(dvA * day7)